# <center> Data to be augmented generator

This part is similar to the starting part of Failure detection and failure classification notebooks. tHe only difference is here we are considering normal operation data as well, so what we'll be doing in Data Augmentation using VAE file is that we'll oversample the failure data of all classes up too the level of normal class. THen, we'll select only best required number of samples!


- For detailed comments, refer to other three notebooks.

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline  
#will make plot outputs appear and be stored within the notebook.
import random
import tensorflow as tf
from tensorflow import keras

import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
SEED = 42

os.environ['PYTHONHASHSEED'] =str(SEED)
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [3]:
from sklearn.preprocessing import RobustScaler, MinMaxScaler
scaler = MinMaxScaler()

def get_normalized_final_dataset():
    
    df = pd.read_excel('New_data_combined.xlsx') # Dataset is now stored in a Pandas Dataframe#
    df = df.sample(df.shape[0])   #shuffling
    df.reset_index(drop = True , inplace = True)
#     df = df.drop_duplicates()   #dropping duplicates
    
    cols_to_drop = ['A1_Pin' , 'A1_Pout' , 'A3_Pin' ,'A3_Pout' , 'A4_Pin', 'A4_Pout','CD', 'Q-Factor' , 'A2_Pout' ]
    df.drop(cols_to_drop ,axis = 'columns', inplace = True)
    
#     cols_to_scale = ['OSNR', 'BER','A2_Pin' ]
#     df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])
    
    return df

In [4]:
dataset = get_normalized_final_dataset()

In [5]:
dataset.Failure_Type.value_counts()

0    29998
4     1000
1     1000
2      999
3      999
5      999
Name: Failure_Type, dtype: int64

In [6]:
dataset.drop_duplicates(inplace = True)

In [7]:
dataset.Failure_Type.value_counts()

0    2514
3     991
5     325
4     257
1     224
2     152
Name: Failure_Type, dtype: int64

In [8]:
from sklearn.model_selection import train_test_split

def get_train_test_validation_data(df):
    
    X = df.drop('Failure_Type', axis = 'columns')
    y = df['Failure_Type']
    
    X_train_val , X_test, y_train_val, y_test = train_test_split (X, y , test_size = 0.30, random_state = SEED , stratify = y)
    X_train , X_val, y_train, y_val = train_test_split (X_train_val, y_train_val , test_size = 0.30, random_state = SEED , stratify = y_train_val)
    
    return X_train, y_train, X_test, y_test, X_val, y_val

In [9]:
X_train, y_train, X_test, y_test, X_val, y_val = get_train_test_validation_data(dataset)

In [10]:
X_train.head()

,A2_Pin,OSNR,BER,Alarm
25844,-23.3,23.8,2.200000e-07,0
12958,-24.8,19.9,4.570000e-05,1
2674,-23.7,23.2,6.610000e-07,0
15223,-25.7,20.5,9.590000e-06,1
10744,-23.4,23.4,2.410000e-07,0


In [11]:
X_train.shape

(2186, 4)

In [12]:
y_train.value_counts()

0    1231
3     486
5     159
4     126
1     110
2      74
Name: Failure_Type, dtype: int64

In [13]:
df_train = X_train.copy()
df_train['Failure_Type'] = y_train.values

In [14]:
df_train.head(10)

,A2_Pin,OSNR,BER,Alarm,Failure_Type
25844,-23.3,23.8,2.200000e-07,0,0
12958,-24.8,19.9,4.570000e-05,1,1
2674,-23.7,23.2,6.610000e-07,0,0
15223,-25.7,20.5,9.590000e-06,1,5
10744,-23.4,23.4,2.410000e-07,0,0
1797,-23.7,23.0,7.470000e-07,0,0
7132,-23.7,23.2,7.040000e-07,0,0
627,-23.7,23.1,7.500000e-07,0,0
6555,-24.8,19.8,4.600000e-05,1,1
6292,-30.8,17.0,8.899290e-04,1,3


In [15]:
df_train.to_csv('Training_data_to_be_augmented_final.csv', index = False)